In [16]:
from enum import Enum

class Direction(Enum):
  NORTH = 0
  EAST = 1
  SOUTH = 2
  WEST = 3

  @staticmethod
  def grid_diff(direction) -> tuple[int, int]:
    if direction == Direction.NORTH:
      return -1, 0
    if direction == Direction.EAST:
      return 0, 1
    if direction == Direction.SOUTH:
      return 1, 0
    if direction == Direction.WEST:
      return 0, -1

  @staticmethod
  def turn_clockwise(direction):
    return Direction((direction.value + 1) % 4)
  
  @staticmethod
  def turn_counterclockwise(direction):
    if direction == Direction.NORTH:
      return Direction.WEST
    return Direction(direction.value - 1)

  @staticmethod
  def is_horizontal(direction):
    return direction == Direction.EAST or direction == Direction.WEST

with open('input.txt') as f:
  grid = f.read().splitlines()


def new_beam(i: int, j: int, new_direction: Direction):
  new_i = i + Direction.grid_diff(new_direction)[0]
  new_j = j + Direction.grid_diff(new_direction)[1]
  return new_i, new_j, new_direction

def next_direction(mirror: str, prev_direction: Direction):
  if (mirror == '/' and not Direction.is_horizontal(prev_direction)) \
     or (mirror == '\\' and Direction.is_horizontal(prev_direction)):
    return Direction.turn_clockwise(prev_direction)
  return Direction.turn_counterclockwise(prev_direction)


def energize(starting_beam):
  beam_stats: list[list[list[Direction]]] = [[[] for value in row] for row in grid]
  beams = [starting_beam]
  while beams:
    i, j, direction = beams.pop()
    while True:
      if i < 0 or j < 0 or i >= len(grid) or j >= len(grid[0]):
        break
      if direction in beam_stats[i][j]:
        break
      beam_stats[i][j].append(direction)
      cell = grid[i][j]
      if cell == '.' or \
        (Direction.is_horizontal(direction) and cell == '-') or \
        (not Direction.is_horizontal(direction) and cell == '|'):
        i, j, direction = new_beam(i, j, direction)
        continue
      if cell == '-' or cell == '|':
        beams.append(new_beam(i, j, Direction.turn_clockwise(direction)))
        beams.append(new_beam(i, j, Direction.turn_counterclockwise(direction)))
        break
      i, j, direction = new_beam(i, j, next_direction(cell, direction))

  res = 0
  for row in beam_stats:
    for v in row:
      if len(v) > 0:
        res += 1

  return res

# part 1
print(energize((0, 0, Direction.EAST)))

# part2
results = []
for i in range(len(grid)):
  results.append(energize((0, i, Direction.SOUTH)))
  results.append(energize((len(grid) - 1, i, Direction.NORTH)))
for i in range(len(grid[0])):
  results.append(energize((i, 0, Direction.EAST)))
  results.append(energize((i, len(grid[0]) -1, Direction.WEST)))
print(max(results))

6855
7513
